In [19]:
#Instalamos las librerias
%pip install streamlit


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
#Instalamos la libreria de PLOTY
%pip install plotly

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
#Instalemos libreria para analisis de variables categoricas 
%pip install funpymodeling

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
#para instalar npm en visual studio
#1. Desde Google escribir node.js
#2. Instalar la versión más recomendada 


**Implementación de DASHBOARD**

In [147]:
%%writefile midashboardmexico.py

import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

st.set_page_config(page_title="México Dashboard", layout="wide", page_icon="🌍")

st.markdown(
    """
    <style>
    [data-testid="stSidebar"] {
        background-color: #6A0DAD;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

st.title("Análisis de Datos de México")

st.sidebar.image("147vs.png", width=250)

@st.cache_resource
def load_data():
    df1 = pd.read_csv("mexico_limpio.csv")
    df2 = pd.read_csv("mexico_limpio.csv")
    table = freq_tbl(df1['room_type'])
    Filtro = table[table['frequency'] > 1]
    Filtro_index1 = Filtro.set_index('room_type')
    numeric_df1 = Filtro_index1.select_dtypes(['float', 'int'])
    numeric_cols1 = numeric_df1.columns
    numeric_df2 = df2.select_dtypes(['float', 'int'])
    numeric_cols2 = numeric_df2.columns
    return Filtro_index1, df2, numeric_df1, numeric_cols1, numeric_df2, numeric_cols2

def generate_freq_table(df, column):
    if column in df.columns:
        table = freq_tbl(df[column])
        if isinstance(table, pd.DataFrame):
            if len(table) > 10:
                table = table.head(10)
            return table
        else:
            return None
    else:
        return None

Filtro_index1, df2, numeric_df1, numeric_cols1, numeric_df2, numeric_cols2 = load_data()

st.sidebar.title("Menú Dashboard")
st.sidebar.header("Selecciona la sección de preferencia")
st.sidebar.subheader("Selección de panel")

Frames = st.selectbox(label="Selecciona la Sección", options=["Modelado Explicativo", "Modelado Predictivo", "Visualización de Datos", "Análisis Univariado", "Predicción de Precios"])

if Frames == "Modelado Explicativo":
    st.title("MÉXICO - Modelado Explicativo")
    st.header("Análisis Univariado de Variables")
    variable_type = st.selectbox("Selecciona el tipo de variable:", ["Categórica", "Numérica"])
    check_box = st.sidebar.checkbox(label="Mostrar Dataset")
    if check_box:
        st.write(Filtro_index1)
        st.write(df2)
    Vars_num_selected = st.sidebar.multiselect("Variables Numéricas", numeric_cols2)
    if Vars_num_selected:
        figure1 = px.line(df2, x=df2.index, y=Vars_num_selected, title="Gráfico de Línea", width=1600, height=600)
        st.plotly_chart(figure1)

elif Frames == "Modelado Predictivo":
    st.title("México - Modelado Predictivo")
    st.header("Análisis Univariado")
    Vars_Num = st.selectbox(label="Variables", options=["frequency", "percentage", "cumulative_perc"])
    selected_column = None  
    if Vars_Num == "frequency":
        selected_column = numeric_cols1[0]
    elif Vars_Num == "percentage":
        selected_column = numeric_cols1[1]
    elif Vars_Num == "cumulative_perc":
        selected_column = numeric_cols1[2]
    if selected_column is not None:
        figure2 = px.bar(data_frame=Filtro_index1, x=Filtro_index1.index, y=selected_column, title='Análisis Univariado')
        figure2.update_xaxes(automargin=True)
        figure2.update_yaxes(automargin=True)
        st.plotly_chart(figure2)
    else:
        st.write("Por favor, selecciona una variable del menú desplegable.")

elif Frames == "Visualización de Datos":
    st.title("México - Visualización de Datos")
    st.header("Scatter Plot")
    x_selected = st.sidebar.selectbox(label="Variable X", options=numeric_cols2)
    y_selected = st.sidebar.selectbox(label="Variable Y", options=numeric_cols2)
    figure3 = px.scatter(data_frame=numeric_df2, x=x_selected, y=y_selected, title="Dispersiones")
    st.plotly_chart(figure3)

elif Frames == "Análisis Univariado":
    st.title("Análisis Univariado")
    st.header("Distribución de Variables Categóricas y Numéricas")
    cat_var = st.sidebar.selectbox("Selecciona una variable categórica", options=df2.select_dtypes(include=['object']).columns)
    num_var = st.sidebar.selectbox("Selecciona una variable numérica", options=numeric_cols2)
    st.subheader(f"Distribución de {cat_var}")
    cat_freq_table = generate_freq_table(df2, cat_var)
    if cat_freq_table is not None:
        st.write(cat_freq_table)
        cat_barplot = px.bar(cat_freq_table, x=cat_var, y='frequency', title=f'Frecuencia de {cat_var}', color_discrete_sequence=px.colors.qualitative.Pastel)
        st.plotly_chart(cat_barplot)
    st.subheader(f"Distribución de {num_var}")
    num_histogram = px.histogram(df2, x=num_var, nbins=50, title=f'Distribución de {num_var}', color_discrete_sequence=['#636EFA'])
    st.plotly_chart(num_histogram)

elif Frames == "Predicción de Precios":
    st.title("Predicción de Precios")
    st.header("Modelo de Regresión Lineal Múltiple para Predicción de Precios")
    input_vars = st.sidebar.multiselect("Selecciona las variables de entrada", options=numeric_cols2)
    if len(input_vars) > 1:
        df2_filtered = df2.dropna(subset=input_vars + ['price'])
        X = df2_filtered[input_vars]
        y = df2_filtered['price']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        model = LinearRegression()
        model.fit(X_train, y_train)
        st.subheader("Ingresa los valores para las variables seleccionadas:")
        input_data = {}
        for var in input_vars:
            input_data[var] = st.number_input(f"{var}", value=float(df2[var].mean()))
        input_df = pd.DataFrame([input_data])
        prediction = model.predict(input_df)
        st.write(f"Predicción del Precio: {prediction[0]:.2f}")

        # Mostrar gráfica de predicción
        st.subheader("Gráfica de Predicción")
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
        fig = px.scatter(x=y_train, y=y_pred_train, labels={'x': 'Actual', 'y': 'Predicted'}, title='Actual vs Predicted (Train)')
        fig.add_trace(go.Scatter(x=y_test, y=y_pred_test, mode='markers', name='Test', marker=dict(color='red')))
        st.plotly_chart(fig)
    else:
        st.write("Selecciona al menos dos variables para la regresión múltiple.")

Overwriting midashboardmexico.py
